In [ ]:
import pandas as pd

df = pd.read_csv("B_1.csv")
print(df.columns)

In [ ]:
for i, c in enumerate(df.columns, start=1):
    print(f"{i:03d}  {c}")

In [ ]:
import pandas as pd
import os

folder = "."          # change if files are in another folder
N = 50                # first 50 buildings: B_0 ... B_49

results = []

for i in range(N):
    file = os.path.join(folder, f"B_{i}.csv")
    df = pd.read_csv(file)

    # Heating demand (thermal)
    df["Heating_Demand_kWh"] = df["Qhs_kWh"]

    total_heat = df["Heating_Demand_kWh"].sum()      # total heating over all rows
    peak_heat  = df["Heating_Demand_kWh"].max()      # peak per timestep (if timestep is hourly, it's kWh per hour)

    results.append({
        "building": f"B_{i}",
        "total_heating_kWh": total_heat,
        "peak_timestep_kWh": peak_heat
    })

summary = pd.DataFrame(results)
print(summary)

# Save summary table
summary.to_csv("HeatingDemand_summary_B0_to_B49.csv", index=False)


In [ ]:
import pandas as pd

df = pd.read_csv("nom et electricite.csv")
print(df.columns)
df


In [ ]:
for i, c in enumerate(df.columns, start=1):
    print(f"{i:03d}  {c}")


In [ ]:
import pandas as pd
import os

# 1) Load the list from QGIS/Excel export
elec_df = pd.read_csv("nom et electricite.csv")

# Check columns (optional)
print(elec_df.columns)

# 2) Keep only buildings where heating is electricity
# (adjust the string if your file uses "Electricity", "electricite", "ELECTRIC", etc.)
electric_buildings = (
    elec_df.loc[elec_df["heating_en"].str.lower().str.contains("electric", na=False), "name"]
    .astype(str)
    .str.strip()
    .tolist()
)

print("Electric-heated buildings:", len(electric_buildings))
print(electric_buildings[:10])


In [ ]:
import pandas as pd

ref_path = "nom et electricite.csv"
ref = pd.read_csv(ref_path, sep=None, engine="python")

wanted = {f"B_{i}" for i in range(0, 601)}  # 0..600 inclusive

selected = ref[ref["name"].astype(str).str.strip().isin(wanted)].copy()

print("Buildings found:", selected["name"].nunique())
selected.to_csv("selected_buildings_B0_to_B600.csv", index=False)
print("Saved: selected_buildings_B0_to_B600.csv")


In [ ]:
# Unique building names, sorted naturally by the number after B_
names = selected["name"].astype(str).str.strip().unique().tolist()

# sort by numeric part (B_123 -> 123)
names_sorted = sorted(names, key=lambda x: int(x.split("_")[1]))

print("Count:", len(names_sorted))
print(names_sorted)


In [ ]:
from pathlib import Path
import pandas as pd

# 1) define buildings (use names_sorted if you have it, otherwise build from selected)
try:
    buildings = names_sorted
except NameError:
    buildings = (selected["name"].astype(str).str.strip().drop_duplicates().tolist())
    buildings = sorted(buildings, key=lambda x: int(x.split("_")[1]))

data_folder = Path("D:/Analysing/Analysation")

all_qhs = []
missing_files = []
missing_column = []
read_ok = 0

for b in buildings:
    f = data_folder / f"{b}.csv"

    if not f.exists():
        missing_files.append(b)
        continue

    df = pd.read_csv(f, sep=None, engine="python")
    read_ok += 1

    # IMPORTANT: exact column name (case sensitive)
    if "Qhs_kWh" not in df.columns:
        missing_column.append(b)
        continue

    tmp = df[["Qhs_kWh"]].copy()
    tmp["building"] = b
    all_qhs.append(tmp[["building", "Qhs_kWh"]])

print("Buildings requested:", len(buildings))
print("Files read OK:", read_ok)
print("Missing files:", len(missing_files), "example:", missing_files[:10])
print("Missing Qhs_kWh column:", len(missing_column), "example:", missing_column[:10])

# If nothing collected, show columns from ONE existing file to debug
if len(all_qhs) == 0:
    existing = next((data_folder / f"{b}.csv" for b in buildings if (data_folder / f"{b}.csv").exists()), None)
    if existing:
        df0 = pd.read_csv(existing, sep=None, engine="python")
        print("Example file used for debug:", existing.name)
        print("Columns in that file:")
        print(df0.columns.tolist())
    else:
        print("No building files found at all in:", data_folder.resolve())
else:
    qhs_long = pd.concat(all_qhs, ignore_index=True)
    qhs_long


In [ ]:
from pathlib import Path
import pandas as pd

buildings = names_sorted          # your 301 buildings
data_folder = Path("D:/Analysing/Analysation") # folder with B_*.csv

total_by_date = None
missing_files = []

for b in buildings:
    f = data_folder / f"{b}.csv"
    if not f.exists():
        missing_files.append(b)
        continue

    df = pd.read_csv(f, sep=None, engine="python")[["date", "Qhs_kWh"]].copy()
    df["date"] = pd.to_datetime(df["date"], errors="coerce")
    df["Qhs_kWh"] = pd.to_numeric(df["Qhs_kWh"], errors="coerce").fillna(0)

    # sum within building per date (just in case duplicates)
    s = df.groupby("date")["Qhs_kWh"].sum()

    if total_by_date is None:
        total_by_date = s
    else:
        total_by_date = total_by_date.add(s, fill_value=0)

print("Missing files:", len(missing_files), "example:", missing_files[:5])

# final dataframe time series
qhs_total_ts = total_by_date.sort_index().rename("Qhs_total_kWh").reset_index()

qhs_total_ts


In [ ]:
wanted_buildings = ["B_4", "B_9"]  # add more here
data_folder = Path("D:/Analysing/Analysation")

dfs = []
for b in wanted_buildings:
    f = data_folder / f"{b}.csv"
    df = pd.read_csv(f, sep=None, engine="python")[["date", "Qhs_kWh"]].copy()
    df["date"] = pd.to_datetime(df["date"], errors="coerce")
    df["Qhs_kWh"] = pd.to_numeric(df["Qhs_kWh"], errors="coerce")
    df = df.rename(columns={"Qhs_kWh": b})
    dfs.append(df)

qhs_two = dfs[0].merge(dfs[1], on="date", how="outer").sort_values("date").reset_index(drop=True)
qhs_two


In [ ]:
# sum across building columns (ignore the date column)
qhs_two["Qhs_sum_kWh"] = qhs_two[["B_4", "B_9"]].sum(axis=1, skipna=True)

qhs_two


In [ ]:
from pathlib import Path
import pandas as pd

wanted_buildings = names_sorted                 # <- your 301 buildings list
data_folder = Path("D:/Analysing/Analysation")  # <- folder with B_*.csv

dfs = []
missing = []

for b in wanted_buildings:
    f = data_folder / f"{b}.csv"
    if not f.exists():
        missing.append(b)
        continue

    df = pd.read_csv(f, sep=None, engine="python")[["date", "Qhs_kWh"]].copy()
    df["date"] = pd.to_datetime(df["date"], errors="coerce")
    df["Qhs_kWh"] = pd.to_numeric(df["Qhs_kWh"], errors="coerce")

    # rename heating column to building name
    df = df.rename(columns={"Qhs_kWh": b})

    dfs.append(df)

print("Loaded:", len(dfs))
print("Missing:", len(missing), "example:", missing[:10])

# merge all buildings on date
qhs_all = dfs[0]
for d in dfs[1:]:
    qhs_all = qhs_all.merge(d, on="date", how="outer")

qhs_all = qhs_all.sort_values("date").reset_index(drop=True)

qhs_all


In [ ]:
# all building columns (everything except date)
building_cols = qhs_all.columns.drop("date")

qhs_all["Qhs_sum_kWh"] = qhs_all[building_cols].sum(axis=1, skipna=True)

qhs_all


In [ ]:
# pip install openpyxl  # run once if needed

qhs_all.to_excel("Qhs_timeseries_all_buildings.xlsx", index=False)
print("Saved: Qhs_timeseries_all_buildings.xlsx")


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

path = "Qhs_timeseries_all_buildings.xlsx"
df = pd.read_excel(path)
df["date"] = pd.to_datetime(df["date"])

building_cols = [c for c in df.columns if c.startswith("B_")]

plt.figure()
for b in building_cols:
    plt.plot(df["date"], df[b])  # no legend (too many)

plt.xlabel("date")
plt.ylabel("Qhs_kWh")
plt.title("Heating demand time series (all buildings)")
plt.show()


In [ ]:
import pandas as pd
import plotly.graph_objects as go

path = "Qhs_timeseries_all_buildings.xlsx"
df = pd.read_excel(path)
df["date"] = pd.to_datetime(df["date"])

building_cols = [c for c in df.columns if c.startswith("B_")]

fig = go.Figure()
for b in building_cols:
    fig.add_trace(go.Scatter(
        x=df["date"],
        y=df[b],
        mode="lines",
        name=b,
        showlegend=False  # legend would be unusable with 301 items
    ))

Qhs_smooth = df["Qhs_sum_kWh"].rolling(24, center=True).mean()

fig.update_layout(
    title="Heating demand time series (all buildings)",
    xaxis_title="Date",
    yaxis_title="Qhs_kWh",
    hovermode="x unified"
)

fig.show()


In [ ]:
import pandas as pd
import plotly.graph_objects as go

# --- 1) HEATING (from your excel) ---
heat_path = "Qhs_timeseries_all_buildings.xlsx"   # your heating file
heat = pd.read_excel(heat_path)
heat["date"] = pd.to_datetime(heat["date"], errors="coerce")
heat = heat[["date", "Qhs_sum_kWh"]].dropna(subset=["date"]).sort_values("date")

# --- 2) GRID (from Total_demand_hourly.csv) ---
grid = pd.read_csv("Total_demand_hourly.csv", sep=None, engine="python")
grid["date"] = pd.to_datetime(grid["date"], errors="coerce")
grid["GRID_kWh"] = pd.to_numeric(grid["GRID_kWh"], errors="coerce")
grid = grid[["date", "GRID_kWh"]].dropna(subset=["date"]).sort_values("date")

# --- 3) MERGE + COMBINE ---
df = heat.merge(grid, on="date", how="inner")
df["GRID_plus_heating_kWh"] = df["GRID_kWh"] + df["Qhs_sum_kWh"]

# --- 4) PLOT (Plotly) ---
fig = go.Figure()
fig.add_trace(go.Scatter(x=df["date"], y=df["GRID_kWh"], mode="lines", name="GRID_kWh"))
fig.add_trace(go.Scatter(x=df["date"], y=df["Qhs_sum_kWh"], mode="lines", name="Heating (Qhs_sum_kWh)"))
fig.add_trace(go.Scatter(x=df["date"], y=df["GRID_plus_heating_kWh"], mode="lines", name="GRID + Heating"))

fig.update_layout(
    title="Grid demand, heating demand, and combined demand (hourly)",
    xaxis_title="Date",
    yaxis_title="kWh",
    hovermode="x unified"
)
fig.show()


In [ ]:
import pandas as pd
import plotly.graph_objects as go

# --- 1) HEATING ---
heat = pd.read_excel("Qhs_timeseries_all_buildings.xlsx")
heat["date"] = pd.to_datetime(heat["date"], errors="coerce")
heat["Qhs_sum_kWh"] = pd.to_numeric(heat["Qhs_sum_kWh"], errors="coerce")
heat = heat[["date", "Qhs_sum_kWh"]].dropna(subset=["date"]).sort_values("date")

# --- 2) GRID ---
grid = pd.read_csv("Total_demand_hourly.csv", sep=None, engine="python")
grid["date"] = pd.to_datetime(grid["date"], errors="coerce")
grid["GRID_kWh"] = pd.to_numeric(grid["GRID_kWh"], errors="coerce")
grid = grid[["date", "GRID_kWh"]].dropna(subset=["date"]).sort_values("date")

# --- 3) MERGE (keep all hours) ---
df = pd.merge(heat, grid, on="date", how="outer").sort_values("date")

# Fill missing values with 0 (so sum works)
df["Qhs_sum_kWh"] = df["Qhs_sum_kWh"].fillna(0)
df["GRID_kWh"] = df["GRID_kWh"].fillna(0)

# --- 4) ADD THEM ---
df["GRID_plus_heating_kWh"] = df["GRID_kWh"] + df["Qhs_sum_kWh"]

# --- 5) PLOT ---
fig = go.Figure()
fig.add_trace(go.Scatter(x=df["date"], y=df["GRID_kWh"], mode="lines", name="GRID_kWh"))
fig.add_trace(go.Scatter(x=df["date"], y=df["Qhs_sum_kWh"], mode="lines", name="Heating (Qhs_sum_kWh)"))
fig.add_trace(go.Scatter(x=df["date"], y=df["GRID_plus_heating_kWh"], mode="lines", name="GRID + Heating"))

fig.update_layout(
    title="Grid demand, heating demand, and combined demand (hourly)",
    xaxis_title="Date",
    yaxis_title="kWh",
    hovermode="x unified"
)
fig.show()


i have use COP (coefficient of performance) assumption

In [ ]:
import pandas as pd
import plotly.graph_objects as go

COP = 4 # choose 2.5–4; use what prof recommends (3 is common)

# --- HEATING (thermal) ---
heat = pd.read_excel("Qhs_timeseries_all_buildings.xlsx")
heat["date"] = pd.to_datetime(heat["date"], errors="coerce")
heat["Qhs_sum_kWh"] = pd.to_numeric(heat["Qhs_sum_kWh"], errors="coerce")
heat = heat[["date", "Qhs_sum_kWh"]].dropna(subset=["date"]).sort_values("date")

# Convert thermal heating -> electric heating
heat["HEAT_elec_kWh"] = heat["Qhs_sum_kWh"] / COP

# --- BASE ELECTRICITY (grid) ---
grid = pd.read_csv("Total_demand_hourly.csv", sep=None, engine="python")
grid["date"] = pd.to_datetime(grid["date"], errors="coerce")
grid["GRID_kWh"] = pd.to_numeric(grid["GRID_kWh"], errors="coerce")
grid = grid[["date", "GRID_kWh"]].dropna(subset=["date"]).sort_values("date")

# --- MERGE ---
df = pd.merge(heat, grid, on="date", how="outer").sort_values("date")
df[["Qhs_sum_kWh", "HEAT_elec_kWh", "GRID_kWh"]] = df[["Qhs_sum_kWh", "HEAT_elec_kWh", "GRID_kWh"]].fillna(0)

# --- FEEDER DEMAND (electric) ---
df["FEEDER_kWh"] = df["GRID_kWh"] + df["HEAT_elec_kWh"]
df["FEEDER_MWh"] = df["FEEDER_kWh"] / 1000

# --- PLOT ---
fig = go.Figure()
fig.add_trace(go.Scatter(x=df["date"], y=df["FEEDER_MWh"], mode="lines",
                         name=f"Simulated feeder demand (MWh), COP={COP}"))

fig.update_layout(
    title="Base + Electric Heating",
    xaxis_title="Date",
    yaxis_title="MWh per hour",
    hovermode="x unified"
)
fig.show()
